In [22]:
%load_ext autoreload
%autoreload 2 
import os
import sys
import re

sys.path.append("../../bas-apres/")
import apres as ap

sys.path.append("../../xapres/")
import xapres as xa
import numpy as np
import gcsfs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
# get a random remote dat files and download it
directory=root = "gs://ldeo-glaciology/apres/thwaites/continuous/ApRES_LTG"

fs = xa.load.from_dats()
remote_dat = fs.list_files(directory)[20]
filesystem = gcsfs.GCSFileSystem()
filesystem.get(rpath = remote_dat, lpath = '../tests/')
local_file = '../tests/DATA2023-02-16-0437.DAT'

Open it to see what shape the data is

In [77]:
using_xapres = xa.load.generate_xarray('data', computeProfiles=False)

100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


In [78]:
using_xapres

<xarray.Dataset>
Dimensions:          (time: 2, chirp_time: 40001, chirp_num: 100,
                      attenuator_setting_pair: 1)
Coordinates:
  * time             (time) datetime64[ns] 2023-02-16T04:37:28 2023-02-17T04:...
  * chirp_time       (chirp_time) float64 0.0 2.5e-05 5e-05 ... 1.0 1.0 1.0
  * chirp_num        (chirp_num) int64 0 1 2 3 4 5 6 7 ... 93 94 95 96 97 98 99
    filename         (time) <U28 'data/DATA2023-02-16-0437.DAT' 'data/DATA202...
    burst_number     (time) int64 0 1
    AFGain           (attenuator_setting_pair) int64 -4
    attenuator       (attenuator_setting_pair) float64 22.0
    orientation      (time) <U7 'unknown' 'unknown'
Dimensions without coordinates: attenuator_setting_pair
Data variables:
    chirp            (time, chirp_time, chirp_num, attenuator_setting_pair) float64 ...
    latitude         (time) float64 0.0 0.0
    longitude        (time) float64 0.0 0.0
    battery_voltage  (time) float64 12.39 12.39
    temperature_1    (time) float64 -18.35 -12.28
    temperature_2    (time) float64 -10.52 -7.773
Attributes:
    constants:   {'c': 300000000.0, 'K': 200000000.0, 'f_1': 200000000, 'f_2'...
    processing:  Created on 2024-11-12 16:18:10.114351

Open the dat file from local and from the bucket and check they are the same

In [70]:
with ap.ApRESFile(local_file) as f_local: 
    f_local.read()
print(len(f.bursts))


2


In [71]:
with ap.ApRESFile(remote_dat) as f_remote: 
    f_remote.read()
print(len(f.bursts))


2


In [72]:
all( (f_local.bursts[n].data == f_remote.bursts[n].data).all() for n in range(len(f_remote.bursts)) )

True

In [56]:
(remote_data == local_data).all()

True

Below are notes that helped to edit bas-apres to get it to work with remote dats.

The next three cells are about loading the actual data. 

In [65]:
import os
import numpy as np
filesystem = gcsfs.GCSFileSystem()

#path = local_file
path = remote_dat
count = 1600040


In [66]:
# remote
fp = filesystem.open(remote_dat, mode = 'rb')
file_size = filesystem.info(path)['size']
inbuff = fp.read(count * 2)
out = (np.frombuffer(inbuff, dtype=np.uint16))#/2**16*2.5-1.25
out

array([ 2573, 10794,  8234, ..., 33839, 33823, 33805], dtype=uint16)

In [67]:
# local
fp = open(local_file, mode = 'r', encoding='latin-1')
file_size = os.fstat(fp.fileno()).st_size
print(fp.tell())
out  = np.fromfile(fp, dtype=np.dtype('<u2'), count=count)
out


0


array([ 2573, 10794,  8234, ..., 33839, 33823, 33805], dtype=uint16)

The next two cells are about loading the header

In [68]:
# remote
fp = filesystem.open(remote_dat, mode = 'r', encoding='latin-1')

data_start = -1
header_lines = []
header_start = 0
fp.seek(header_start, 0)
line = fp.readline()
header_lines.append(line.rstrip())
while(line):
        # The data section follows the end of header marker
        if re.match( r'\*\*\* End Header', line):
            data_start = fp.tell()
            break

        line = fp.readline()
        header_lines.append(line.rstrip())
header_lines

['',
 '*** Burst Header ***',
 'Time stamp=2023-02-16 04:37:28',
 'RMB_Issue=2c',
 'VAB_Issue=3a',
 'SW_Issue=104.0',
 'Venom_Issue=20180522',
 'Alternate=0',
 'MonoTx=1',
 'MonoRx=1',
 'NSubBursts=100',
 'NData=0',
 'NSAFData=0',
 'Triples=0,0,0,0,0,0,0,0,0,0,0,0',
 'Average=0',
 'RepSecs=86400',
 'CheckEthernet=1',
 'N_ADC_SAMPLES=40001',
 'MAX_DATA_FILE_LENGTH=10000000',
 'MAX_SAF_FILE_LENGTH=0',
 'ANTENNA_SELECT=0',
 'nAttenuators=1',
 'Housekeeping=1',
 'GPSon=0',
 'SyncGPS=1',
 'Iridium=0',
 'WATCHDOG_TASK_SECS=3600',
 'IntervalMode=0',
 'InterChirpDelay=50',
 'Attenuator1=22,30,30,30',
 'AFGain=-4,-14,-14,-14',
 'TxAnt=1,0,0,0,0,0,0,0',
 'RxAnt=1,0,0,0,0,0,0,0',
 'maxDepthToGraph=2200',
 'SleepMode=0',
 'LogOn=1',
 'Reg00="00000008"',
 'Reg01="000C0820"',
 'Reg02="0D1F41C8"',
 'Reg0B="6666666633333333"',
 'Reg0C="000053E3000053E3"',
 'Reg0D="186A186A"',
 'Reg0E="08B5000000000000"',
 'SamplingFreqMode=0',
 'Settle_Cycles=0',
 'BatteryCheck=11.2,11.7',
 'ER_ICE=3.18',
 'GPS_TIMEOU

In [69]:
# local
fp = open(local_file, mode = 'r', encoding='latin-1')

data_start = -1
header_lines = []
header_start = 0
fp.seek(header_start, 0)
line = fp.readline()
header_lines.append(line.rstrip())
while(line):
        # The data section follows the end of header marker
        if re.match( r'\*\*\* End Header', line):
            data_start = fp.tell()
            break

        line = fp.readline()
        header_lines.append(line.rstrip())

header_lines

['',
 '*** Burst Header ***',
 'Time stamp=2023-02-16 04:37:28',
 'RMB_Issue=2c',
 'VAB_Issue=3a',
 'SW_Issue=104.0',
 'Venom_Issue=20180522',
 'Alternate=0',
 'MonoTx=1',
 'MonoRx=1',
 'NSubBursts=100',
 'NData=0',
 'NSAFData=0',
 'Triples=0,0,0,0,0,0,0,0,0,0,0,0',
 'Average=0',
 'RepSecs=86400',
 'CheckEthernet=1',
 'N_ADC_SAMPLES=40001',
 'MAX_DATA_FILE_LENGTH=10000000',
 'MAX_SAF_FILE_LENGTH=0',
 'ANTENNA_SELECT=0',
 'nAttenuators=1',
 'Housekeeping=1',
 'GPSon=0',
 'SyncGPS=1',
 'Iridium=0',
 'WATCHDOG_TASK_SECS=3600',
 'IntervalMode=0',
 'InterChirpDelay=50',
 'Attenuator1=22,30,30,30',
 'AFGain=-4,-14,-14,-14',
 'TxAnt=1,0,0,0,0,0,0,0',
 'RxAnt=1,0,0,0,0,0,0,0',
 'maxDepthToGraph=2200',
 'SleepMode=0',
 'LogOn=1',
 'Reg00="00000008"',
 'Reg01="000C0820"',
 'Reg02="0D1F41C8"',
 'Reg0B="6666666633333333"',
 'Reg0C="000053E3000053E3"',
 'Reg0D="186A186A"',
 'Reg0E="08B5000000000000"',
 'SamplingFreqMode=0',
 'Settle_Cycles=0',
 'BatteryCheck=11.2,11.7',
 'ER_ICE=3.18',
 'GPS_TIMEOU